In [3]:
%run Model.ipynb

nadh_c + succoa_c --> coa_c + h_c + nad_c + succinyl_semialdehyde_c
nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c
accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c
four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c
four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c
akg_c --> co2_c + succinyl_semialdehyde_c
nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c
accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c
four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c
four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c


# Co-Factor Swap attempt 

In [5]:
from cameo import models
model_orig = model_sucd #importing our model

In [6]:
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
from cameo.strain_design.heuristic.evolutionary.objective_functions import biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva

In [7]:
model = model_orig.copy() # create a copy for changing the metobolites and reactions

## Biomass yield

In [11]:
solution = model.optimize()
g_dw_biomass_per_mmol_glucose = solution.objective_value / (-1. * solution.fluxes['EX_glc__D_e'])
print ('Biomass yield=', g_dw_biomass_per_mmol_glucose)

Biomass yield= 0.08769972144269841


In [6]:
for rid in ['SUCD', 'FHBD', 'HBCT', 'HBCR','ADH' ]: # we insert our reactions in our model
    
    model.reactions.get_by_id(rid).knock_out() #checking the reactions and replace them with a better yield
model.reactions.ADH.lower_bound = 0
model.reactions.EX_glc__D_e.lower_bound = -10
model.reactions.EX_o2_e.lower_bound = -10


In [7]:
model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.1 # we set a lower bounder for the biomass

# Calculation of yield in cmol

In [8]:
model.objective = model.reactions.EX_glc__D_e
(model.optimize().objective_value * 4) / (model.reactions.EX_glc__D_e.flux * 6) #the yield is calculated in cmol 

0.6666666666666666

In [9]:
py = product_yield(model.reactions.EX_bdo, model.reactions.EX_glc__D_e) # product's yield
optswap = CofactorSwapOptimization(model=model, objective_function=py)

In [10]:
optswap.run(max_evaluations=2000, max_size=2) # results of cofactor swap ??

Starting optimization at Sat, 20 Nov 2021 11:50:24


HBox()

Finished after 00:01:34


,index,targets,fitness


# Calculations of yields before and after the swap

In [11]:
optswap.model.reactions.EX_bdo.model = optswap.model
optswap.model.objective = optswap.model.reactions.EX_bdo
original = (optswap.model.optimize().objective_value * 4) / (-optswap.model.reactions.EX_glc__D_e.flux * 6) # calculaates the yield before the substrate swap
with TimeMachine() as tm:
    optswap.model.swap_reaction('SUCD', tm)
    swapped = (optswap.optimize().objective_value * 4) / (-optswap.model.reactions.EX_glc__D_e.flux * 6) # calculaates the yield after the substrate swap

print("product/substrate yield without swap: {}\nproduct/substrate yield with swap: {}".format(original, swapped))

AttributeError: can't set attribute